In [1]:
# Realistically, I would need a fail-save for when a dataset has weird values, like throwing that molecule out entirely.

from grape.datasets import QM9
%time dataset = QM9(log=False)


CPU times: user 1min 20s, sys: 621 ms, total: 1min 21s
Wall time: 1min 21s


In [2]:
print(len(dataset))

128096
